# Snelheidsborden Detectie

In [ ]:
# Het importeren van de juiste libraries
import os
import xml.etree.ElementTree as ET
import pandas as pd
from IPython.display import display
from PIL import Image

Een van de obstakels die de kart moet kunnen detecteren en op reageren is de snelheidsborden. Aangezien er nog geen data is gemaakt op de baan in Lelystad maken wij eerst gebruik van een dataset die op het internet is gevonden. Het is belangrijk om als voorbereiding van het trainen van het model een map en een bestand klaar te maken. Zo is het belangrijk dat er een bestand is met alle namen van de labels, en een map met alle foto's. Later gaan we gebruik maken van onze eigen data, maar omdat er nu gebruik wordt gebruikt van externe data is er aan ChatGPT gevraagd een functie te schrijven zodat de bestanden goed klaar gemaakt kunnen worden. Dit zal schelen in tijd zodat in ieder geval de modellen getraind kunnen worden.

In [ ]:
# (ChatGPT, 2024, Promt 1: XML Files to DataFrame)
def extract_names_from_xml(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    names = []
    filename = root.find('filename').text
    for obj in root.findall('.//object'):
        name_elem = obj.find('name')
        if name_elem is not None:
            names.append((filename, name_elem.text))
    return names

def process_directory(directory):
    all_names = []
    for root, _, files in os.walk(directory):
        for file in files:
            if file.endswith('.xml'):
                xml_file = os.path.join(root, file)
                names = extract_names_from_xml(xml_file)
                all_names.extend(names)
    return all_names

def create_dataframe(names):
    df = pd.DataFrame(names, columns=['filename', 'object_name'])
    return df

# De verwijzing naar de map met bestandsnamen
directory_path = r'C:\Users\bosch\Downloads\archive\annotations'
names = process_directory(directory_path)
df = create_dataframe(names)
print(df)

In het DataFrame df staan nu de bestanden met de namen van de categoriën behorend bij de bestanden.

In [ ]:
df.isna().sum()

Wanneer er gekeken wordt naar de inhoud van het DataFrame zijn er geen missende waardes aanwezig.

In [ ]:
df["object_name"].value_counts()

Het is voor het detecteren van snelheidsborden alleen belangrijk dat de foto's worden gebruikt die geclassificeerd zijn als "speedlimit". In de map met foto's is duidelijk de meerderheid van de bestanden een snelheidsbord. Dit zullen de foto's zijn die gebruikt worden voor het uiteindelijke model. Het doel voor dit model is het kunnen aflezen van de getallen die op de verkeersborden staan.

In [ ]:
df_speed = df[df["object_name"] == "speedlimit"]
display(df_speed.head())
len(df_speed)

Nu is het de bedoeling om de map te importeren waar alle foto's in te vinden zijn. Hierna kunnen we de foto's selecteren uit de map die snelheidsborden bevatten. Daarna is onze data klaar om mee te trainen.

In [ ]:
# Path to the directory containing the photos
dir_path = r"C:\Users\bosch\Downloads\archive\images"

# Filter the DataFrame to get only the filenames
photo_filenames = df_speed['filename'].tolist()

# Load and display the photos
for filename in photo_filenames:#[:5]:
    file_path = os.path.join(dir_path, filename)
    if os.path.isfile(file_path):
        image = Image.open(file_path)
        display(image)

Nu is er een bestand gemaakt met alle afbeeldingen die een snelheidsbord bevatten. In de cel hieronder wordt gecheckt of het aantal afbeeldingen uit het df_speed DataFrame overeen komen met het aantal afbeeldingen in het photo_filenames DataFrame.

In [ ]:
print("Length of photo_filenames:", len(photo_filenames))
print("Length of df_speed:", len(df_speed))

In [ ]:
photo_filenames = pd.DataFrame(photo_filenames, columns=["filename"])
photo_filenames

Er is nu een dataframe gemaakt met alle foto's die snelheidsborden bevatten, maar hier zijn nog geen labels aan toegevoegd. Het model kan dus niet weten welke foto's geclassificeerd worden als welke snelheid.

In [ ]:
#photo_filenames.to_csv(r'C:\Users\bosch\Downloads\archive\filenames.csv', index=False)

In de cel hierboven is het bestand opgeslagen met alle namen van de foto's die een snelheidsbord bevatten. Zo kan deze makkelijk terug gevonden en mogelijke labels worden toegevoegd.

### Literatuurlijst
- ChatGPT. (2024), Prompt 1: XML Files to DataFrame. https://chat.openai.com/share/d7e047f4-4bc6-4311-96cc-c5023b0f5cd8